# Week 2

Exo 1

In [1]:
# Initialize a network
from random import random
def initialize_network(n_inputs , n_hidden , n_outputs):
  network = list()
  hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)] 
  network.append(hidden_layer)
  output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)] # output layer should be defined here
  network.append(output_layer)#output layer should be added to the network here
  return network

Exo2

In [2]:
from random import seed
seed(703185)

In [3]:
initialize_network(2,3,1)

[[{'weights': [0.9891032530590325, 0.0265960175075014, 0.5796018782649225]},
  {'weights': [0.3725427863726888, 0.5542717635140079, 0.6006422923773905]},
  {'weights': [0.3568641403648303, 0.5616733148741453, 0.7145775125959037]}],
 [{'weights': [0.9328072439242623,
    0.4400213857630133,
    0.015970702885971688,
    0.24748356156372875]}]]

Network : hidden_layer composé de 3 n_hidden neurons , eux même composé 3 n_inputs+1 weights. Output_layer composé d'1 n_output neuron, lui même composé de 4 n_hidden + 1 weights. 

EXO 3

In [4]:
# Calculate neuron activation for an input
#for the hidden layer, input can be a row from training set, and for output layer, it's the output of each neuron in the hidden layer
def activate(weights, inputs):
  activation = weights[-1]
  for i in range(len(inputs)):
    activation += (weights[i] * inputs[i]) 
  return activation

EXO 4

In [5]:
#Here we proceed to the transfer of the activation function to see what is the output of the neuron. Different transfer function are possibel. Here we will use
#the sigmoid function (S shape, alternante between 0 and 1)
from math import exp
def transfer(activation):
  return 1.0/(1.0 + exp(-activation))

EXO 5

In [6]:
# Forward propagate input to a network output
#In this function, we calculate the output of a layer's neuron and then the value is stored and become the input for the next layer, and that
#until the final layer is done
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

EXO 6

In [7]:
#test forward propagation
network = initialize_network(2,3,1)
#the size of the row should be the same as the number of inputs
row=[1, 0]
output = forward_propagate(network, row)
print(network)
#Because the output layer has one neuron, we only get one final output value
#For now the value doesn't make any sense, but we will change that after

[[{'weights': [0.23836526091710708, 0.2414349845019762, 0.7822916156831258], 'output': 0.7351005313146323}, {'weights': [0.4414340101310036, 0.1438043630364002, 0.650789866397262], 'output': 0.7488002606967885}, {'weights': [0.5545772882836295, 0.5066019392894244, 0.10618567280784674], 'output': 0.6594317562461762}], [{'weights': [0.8480340705380638, 0.08919185231320548, 0.9851981503634689, 0.10273410954913431], 'output': 0.8088542067242253}]]


We will now work on back propagate error
Error is calculated bewteen expected output and what we got, and then this error backpropagate the neural network in a way that able us to change the weight.

Two steps : 
Transfer Derivative.
Error Backpropagation.

EXO 7

In [8]:
#First we need to calculate the derivative, the slope of the output of a neuron
def transfer_derivative(output):
  return output * (1.0 - output)

EXO 8

In [9]:
#Then we apply the error backpropagation
#We have to calculate the error for each output neuron
def backward_propagate_error(network, expected):
  for i in reversed(range(len(network))):
    layer = network[i]
    errors = list()
    if i != len(network) -1 :
      for j in range(len(layer)):
        error = 0.0 
        for neuron in network[i + 1]:
          error += (neuron['weights'][j] * neuron['delta'])
        errors.append(error)
    else : 
      for j in range(len(layer)):
        neuron = layer[j]
        errors.append(expected[j] - neuron['output'])
    for j in range(len(layer)):
      neuron = layer[j]
      neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
#You can see that the error signal calculated for each neuron is stored with the name ‘delta’. 
#You can see that the layers of the network are iterated in reverse order, starting at the output 
#and working backwards. This ensures that the neurons in the output layer have ‘delta’ values calculated 
#first that neurons in the hidden layer can use in the subsequent iteration. We chose the name ‘delta’ 
#to reflect the change the error implies on the neuron (e.g. the weight delta).

#You can see that the error signal for neurons in the hidden layer is accumulated from neurons 
#in the output layer where the hidden neuron number j is also the index of the neuron’s weight in the output layer neuron[‘weights’][j].

EXO 9

In [10]:
#test backward_propagate_error
network = initialize_network(2,3,1)
row=[1, 0]
forward_propagate(network, row)
expected=[1]#size of expected is one because one neuron in the output layer
backward_propagate_error(network, expected)
for layer in network:
  print(layer)

[{'weights': [0.2500693907068773, 0.7973015217513506, 0.2399732077762381], 'output': 0.6201164674057597, 'delta': 0.0022254285009697455}, {'weights': [0.4538364070934504, 0.709371778077704, 0.7793992434087644], 'output': 0.7743843869908983, 'delta': 0.0038710403746963257}, {'weights': [0.293906706774522, 0.8936999734100869, 0.337273872602442], 'output': 0.6527571067680923, 'delta': 0.0011091467988885474}]
[{'weights': [0.14021541053419517, 0.32885753479350666, 0.07262897073802954, 0.39613582484477694], 'output': 0.686790484134991, 'delta': 0.0673742844206721}]


Running the example prints the network after the backpropagation of error is complete. You can see that error values (delta) are calculated and stored in the neurons for the output layer and the hidden layer.


EXO 10

In [11]:
#We will now use the error to update the weights
#weight = weight + learning_rate * error * input (weight : given weight, lr : we will specify it later, error : the one we calculated, input : input that caused the error)
def update_weights(network, row, l_rate):
	for i in range(len(network)):
   #input for first layer when i = 0
		inputs = row[:-1]
    #then input of other layer is output of previous layer
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j] #=> weight = weight + learning_rate * error * input
			neuron['weights'][-1] += l_rate * neuron['delta']

#we now have to do it repeatedly

EXO 11

In [12]:
#We create a function that involves looping for a number of epochs and for each epochs update the network
#This function take as parameters an initialized neural network, a training dataset, learning rate, number of epochs and expected number of value
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row) #because forward propagate calculate the output of a neuron
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))]) #formula for the mean square error
			backward_propagate_error(network, expected) #we add delta to the network
			update_weights(network, row, l_rate) #we modify weights
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

EXO 12

In [13]:
#We will now test the complete code with a small dataset
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]

n_inputs = 2 #because we have two variables one target
n_output = 2 #because classification problem with two possible solution
n_hidden = 2 #because we choose to do so
epochs = 20
lr = 0.5

network = initialize_network(n_inputs, n_hidden, n_output)
train_network(network, dataset, lr, epochs, n_output)

>epoch=0, lrate=0.500, error=7.138
>epoch=1, lrate=0.500, error=6.058
>epoch=2, lrate=0.500, error=5.476
>epoch=3, lrate=0.500, error=5.363
>epoch=4, lrate=0.500, error=5.363
>epoch=5, lrate=0.500, error=5.369
>epoch=6, lrate=0.500, error=5.369
>epoch=7, lrate=0.500, error=5.365
>epoch=8, lrate=0.500, error=5.357
>epoch=9, lrate=0.500, error=5.345
>epoch=10, lrate=0.500, error=5.329
>epoch=11, lrate=0.500, error=5.303
>epoch=12, lrate=0.500, error=5.263
>epoch=13, lrate=0.500, error=5.195
>epoch=14, lrate=0.500, error=5.078
>epoch=15, lrate=0.500, error=4.900
>epoch=16, lrate=0.500, error=4.690
>epoch=17, lrate=0.500, error=4.460
>epoch=18, lrate=0.500, error=4.202
>epoch=19, lrate=0.500, error=3.920


In [14]:
for layer in network : 
  print(layer)

[{'weights': [0.8315036465899801, -0.9184614943867463, -0.06290015041559557], 'output': 0.9365526598756515, 'delta': 0.009303188921946488}, {'weights': [0.4079176766138058, 0.9737203488590528, 1.019046018443257], 'output': 0.9998131395665123, 'delta': -6.818419278647324e-06}]
[{'weights': [-1.1044889422365014, 0.2055970442028343, 0.03223667200679335], 'output': 0.33409914062296053, 'delta': -0.07432934272150052}, {'weights': [1.0594151099315132, -0.20765794520283176, 0.0011683949951532824], 'output': 0.6631878277213094, 'delta': 0.07523364495383629}]


We can clearly see that the error (delta) has decreased a lot

EXO 13 - Predict

In [15]:
#For now, we just worked on training de neural network.
#We will now work on a predict function, function that just call the forward_propagate function because it's all we need to make a first prediction
#So this procedure return the index of the output with the bigger probability

def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

EXO 14

In [17]:
#Lets test the predict function
seed(79787)
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
for row in dataset:
	prediction = predict(network, row)
	print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=1
Expected=0, Got=1
Expected=0, Got=1
Expected=0, Got=1
Expected=0, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1


We do have 100% accuracy

EXO 16 - Get the dataset

In [19]:
from csv import reader
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
    #we read the file
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
        #we travel the file, and for each row, we add them to a new list that will be our dataset
			dataset.append(row)
	return dataset

In [21]:
dataset = load_csv('/content/seeds_dataset-2.csv')
dataset
#we clearly have to clean the dataset first

[['15.26\t14.84\t0.871\t5.763\t3.312\t2.221\t5.22\t1'],
 ['14.88\t14.57\t0.8811\t5.554\t3.333\t1.018\t4.956\t1'],
 ['14.29\t14.09\t0.905\t5.291\t3.337\t2.699\t4.825\t1'],
 ['13.84\t13.94\t0.8955\t5.324\t3.379\t2.259\t4.805\t1'],
 ['16.14\t14.99\t0.9034\t5.658\t3.562\t1.355\t5.175\t1'],
 ['14.38\t14.21\t0.8951\t5.386\t3.312\t2.462\t4.956\t1'],
 ['14.69\t14.49\t0.8799\t5.563\t3.259\t3.586\t5.219\t1'],
 ['14.11\t14.1\t0.8911\t5.42\t3.302\t2.7\t5\t1'],
 ['16.63\t15.46\t0.8747\t6.053\t3.465\t2.04\t5.877\t1'],
 ['16.44\t15.25\t0.888\t5.884\t3.505\t1.969\t5.533\t1'],
 ['15.26\t14.85\t0.8696\t5.714\t3.242\t4.543\t5.314\t1'],
 ['14.03\t14.16\t0.8796\t5.438\t3.201\t1.717\t5.001\t1'],
 ['13.89\t14.02\t0.888\t5.439\t3.199\t3.986\t4.738\t1'],
 ['13.78\t14.06\t0.8759\t5.479\t3.156\t3.136\t4.872\t1'],
 ['13.74\t14.05\t0.8744\t5.482\t3.114\t2.932\t4.825\t1'],
 ['14.59\t14.28\t0.8993\t5.351\t3.333\t4.185\t4.781\t1'],
 ['13.99\t13.83\t0.9183\t5.119\t3.383\t5.234\t4.781\t1'],
 ['15.69\t14.75\t0.9058\t5.5

EXO 17

In [22]:
def good_dataset(dataset):
  for i in range(len(dataset)):
    #at first we want to split each line to have several value and not one, and the delimiter
    #here is "\t"
    dataset[i]=dataset[i][0].split("\t")
    for j in range(8):
      #then we convert all the value in float to be useful
      dataset[i][j]=float(dataset[i][j])
      #and then we convert the target to int because sometimes those value are use as index and index doesn't
      #take float value
    dataset[i][7]=int(dataset[i][7])
    dataset[i][7] = dataset[i][7]-1

In [23]:
good_dataset(dataset)
dataset
#and voilaaa

[[15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22, 0],
 [14.88, 14.57, 0.8811, 5.554, 3.333, 1.018, 4.956, 0],
 [14.29, 14.09, 0.905, 5.291, 3.337, 2.699, 4.825, 0],
 [13.84, 13.94, 0.8955, 5.324, 3.379, 2.259, 4.805, 0],
 [16.14, 14.99, 0.9034, 5.658, 3.562, 1.355, 5.175, 0],
 [14.38, 14.21, 0.8951, 5.386, 3.312, 2.462, 4.956, 0],
 [14.69, 14.49, 0.8799, 5.563, 3.259, 3.586, 5.219, 0],
 [14.11, 14.1, 0.8911, 5.42, 3.302, 2.7, 5.0, 0],
 [16.63, 15.46, 0.8747, 6.053, 3.465, 2.04, 5.877, 0],
 [16.44, 15.25, 0.888, 5.884, 3.505, 1.969, 5.533, 0],
 [15.26, 14.85, 0.8696, 5.714, 3.242, 4.543, 5.314, 0],
 [14.03, 14.16, 0.8796, 5.438, 3.201, 1.717, 5.001, 0],
 [13.89, 14.02, 0.888, 5.439, 3.199, 3.986, 4.738, 0],
 [13.78, 14.06, 0.8759, 5.479, 3.156, 3.136, 4.872, 0],
 [13.74, 14.05, 0.8744, 5.482, 3.114, 2.932, 4.825, 0],
 [14.59, 14.28, 0.8993, 5.351, 3.333, 4.185, 4.781, 0],
 [13.99, 13.83, 0.9183, 5.119, 3.383, 5.234, 4.781, 0],
 [15.69, 14.75, 0.9058, 5.527, 3.514, 1.599, 5.046, 0],
 [14

In [24]:
#then we scale the dataset with the minmax formula :
#xi = (xi-min(x))/(max(x)-min(x))
def scale(dataset):
  for i in range(len(dataset)):
    for j in range(7):
      dataset[i][j] = (dataset[i][j]-min(dataset[i][:-1]))/(max(dataset[i][:-1])-min(dataset[i][:-1]))

In [25]:
scale(dataset)

In [26]:
#then we split the dataset into a 80/20 rate
def train_test_split(dataset):
  train_rate = int(0.8*len(dataset))
  train = dataset[:train_rate]
  test = dataset[train_rate:]
  return train, test

In [27]:
train, test = train_test_split(dataset)

EXO 18

In [32]:
#we initialize the network and train our network
network = initialize_network(7,5,3)
train_network(network, train,0.5, 20, 3)

>epoch=0, lrate=0.500, error=100.141
>epoch=1, lrate=0.500, error=50.246
>epoch=2, lrate=0.500, error=51.734
>epoch=3, lrate=0.500, error=51.691
>epoch=4, lrate=0.500, error=51.654
>epoch=5, lrate=0.500, error=51.625
>epoch=6, lrate=0.500, error=51.598
>epoch=7, lrate=0.500, error=51.557
>epoch=8, lrate=0.500, error=51.427
>epoch=9, lrate=0.500, error=51.021
>epoch=10, lrate=0.500, error=51.082
>epoch=11, lrate=0.500, error=51.715
>epoch=12, lrate=0.500, error=51.869
>epoch=13, lrate=0.500, error=51.806
>epoch=14, lrate=0.500, error=51.694
>epoch=15, lrate=0.500, error=51.589
>epoch=16, lrate=0.500, error=51.515
>epoch=17, lrate=0.500, error=51.478
>epoch=18, lrate=0.500, error=51.474
>epoch=19, lrate=0.500, error=51.488


EXO 19

In [33]:
#and finally we try to predict the value from the test set
for row in dataset:
	prediction = predict(network, row)
	print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0, Got=2
Expected=0